<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#import-libraries" data-toc-modified-id="import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>import libraries</a></span></li><li><span><a href="#interest-by-region" data-toc-modified-id="interest-by-region-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>interest by region</a></span><ul class="toc-item"><li><span><a href="#build-payload" data-toc-modified-id="build-payload-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>build payload</a></span><ul class="toc-item"><li><span><a href="#reindex-data" data-toc-modified-id="reindex-data-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>reindex data</a></span></li></ul></li><li><span><a href="#historical-hourly-interest" data-toc-modified-id="historical-hourly-interest-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>historical hourly interest</a></span></li></ul></li><li><span><a href="#Resources" data-toc-modified-id="Resources-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Resources</a></span></li></ul></div>

Resources
1. Detailed run through of python api
https://hackernoon.com/how-to-use-google-trends-api-with-python

2. Article on towards data science
https://towardsdatascience.com/google-trends-api-for-python-a84bc25db88f

In [1]:
# !pip install pytrends

# import libraries

In [1]:
import pandas as pd         
# connect to google
from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US') #pytrends = TrendReq(hl='en-US', tz=360) 

from tqdm.notebook import tqdm

# interest by region

## build payload
i.e. list of key words

In [19]:
help(pytrends.build_payload)

Help on method build_payload in module pytrends.request:

build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='') method of pytrends.request.TrendReq instance
    Create the payload for related queries, interest over time and interest by region



In [5]:
pytrend_geo_codes = {}
# add keys to dictionary https://stackoverflow.com/questions/1024847/how-can-i-add-new-keys-to-a-dictionary

pytrend_geo_codes['Global'] = ''

pytrend_geo_codes['Australia'] = 'AU'
pytrend_geo_codes['New Zealand'] = 'NZ'
pytrend_geo_codes['Singapore'] = 'SG'

pytrend_geo_codes['Canada'] = 'CA'
pytrend_geo_codes['United Kingdom'] = 'GB'
pytrend_geo_codes['United States'] = 'US'

pytrend_geo_codes['France'] = 'FR'
pytrend_geo_codes['Spain'] = 'ES'
pytrend_geo_codes['Japan'] = 'JP'
pytrend_geo_codes['South Korea'] = 'KR'

pytrend_geo_codes['New South Wales'] = 'AU-NSW'
pytrend_geo_codes['Victoria'] = 'AU-VIC'
pytrend_geo_codes['Queensland'] = 'AU-QLD'
pytrend_geo_codes['South Australia'] = 'AU-SA'
pytrend_geo_codes['Western Australia'] = 'AU-WA'
pytrend_geo_codes['Tasmania'] = 'AU-TAS'
pytrend_geo_codes['Northern Territory'] = 'AU-NT'


pytrend_geo_codes

{'Global': '',
 'Australia': 'AU',
 'New Zealand': 'NZ',
 'Singapore': 'SG',
 'Canada': 'CA',
 'United Kingdom': 'GB',
 'United States': 'US',
 'France': 'FR',
 'Spain': 'ES',
 'Japan': 'JP',
 'South Korea': 'KR',
 'New South Wales': 'AU-NSW',
 'Victoria': 'AU-VIC',
 'Queensland': 'AU-QLD',
 'South Australia': 'AU-SA',
 'Western Australia': 'AU-WA',
 'Tasmania': 'AU-TAS',
 'Northern Territory': 'AU-NT'}

In [3]:
# pytrend_geo_codes['Australia']
for key in list(pytrend_geo_codes.keys())[:1]: #.values()
    print(key, pytrend_geo_codes[key])

Australia AU


In [7]:
# build payload

kw_list = ["business cards", 'business card'] # list of keywords to get data 
timeframe = 'today 5-y' #'today 12-m'
df_agg = pd.DataFrame() # DataFrame to store all data
    
for key in tqdm(list(pytrend_geo_codes.keys())[:], desc='Loop through regions'): #.values()
#     print(key, pytrend_geo_codes[key])

    # pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m') 
    # pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='AU') # Australia
    # pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='NZ') # New Zealand
    pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo=pytrend_geo_codes[key]) # New South Wales

    #1 Interest over Time
    data = pytrends.interest_over_time() 
    data = data.reset_index() 

    data.columns = data.columns.str.strip()
    data.columns = data.columns.str.upper()

    # https://www.programiz.com/python-programming/datetime/current-datetime
    from datetime import date
    today = date.today()
#     print("Today's date:", today)

    # put extraction date on table
    data.loc[:, 'DATA EXTRACTED'] = today
    # data.info()
    # data

    # add geo to table
    data.loc[:, 'REGION'] = key
    if key == 'Global':
        data.loc[:, 'GEO'] = 'GLOBAL'
    else:
        data.loc[:, 'GEO'] = pytrend_geo_codes[key]
    

    # delete partial weeks
    data = data[data['ISPARTIAL']=='False']
#     data.info()
#     print(data)
    # add to consolidated DataFrame
    
    df_agg = df_agg.append(data)
    
print(df_agg.info())
print(df_agg.head(2))
print(df_agg.tail(2))

Loop through regions:   0%|          | 0/18 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4680 entries, 0 to 259
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            4680 non-null   datetime64[ns]
 1   BUSINESS CARDS  4680 non-null   int32         
 2   BUSINESS CARD   4680 non-null   int32         
 3   ISPARTIAL       4680 non-null   object        
 4   DATA EXTRACTED  4680 non-null   object        
 5   REGION          4680 non-null   object        
 6   GEO             4680 non-null   object        
dtypes: datetime64[ns](1), int32(2), object(4)
memory usage: 255.9+ KB
None
        DATE  BUSINESS CARDS  BUSINESS CARD ISPARTIAL DATA EXTRACTED  REGION  \
0 2016-12-18              53             78     False     2021-12-17  Global   
1 2016-12-25              43             59     False     2021-12-17  Global   

      GEO  
0  GLOBAL  
1  GLOBAL  
          DATE  BUSINESS CARDS  BUSINESS CARD ISPARTIAL DATA EXTRACTED  \


In [10]:
# process and export data

df_agg.loc[:, 'INDEX'] = df_agg['BUSINESS CARDS']+df_agg['BUSINESS CARD']


df = pd.pivot_table(df_agg, values = 'INDEX', index='DATE', columns = 'GEO').reset_index()
# df.columns
df.to_csv("data/google_trends_bc.csv", index=False)
df = pd.read_csv("data/google_trends_bc.csv")
print(df.info())
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DATE    260 non-null    object
 1   AU      260 non-null    int64 
 2   AU-NSW  260 non-null    int64 
 3   AU-NT   260 non-null    int64 
 4   AU-QLD  260 non-null    int64 
 5   AU-SA   260 non-null    int64 
 6   AU-TAS  260 non-null    int64 
 7   AU-VIC  260 non-null    int64 
 8   AU-WA   260 non-null    int64 
 9   CA      260 non-null    int64 
 10  ES      260 non-null    int64 
 11  FR      260 non-null    int64 
 12  GB      260 non-null    int64 
 13  GLOBAL  260 non-null    int64 
 14  JP      260 non-null    int64 
 15  KR      260 non-null    int64 
 16  NZ      260 non-null    int64 
 17  SG      260 non-null    int64 
 18  US      260 non-null    int64 
dtypes: int64(18), object(1)
memory usage: 38.7+ KB
None
           DATE   AU  AU-NSW  AU-NT  AU-QLD  AU-SA  AU-TAS  AU-VIC  AU-WA  \
0    

### reindex data

In [11]:
import pandas as pd

df = pd.read_csv('data/google_trends_bc.csv')


df_agg = pd.DataFrame()

index_period_start = '2020-12-01'
index_period_end = '2021-11-30'

index_avg = df[df['DATE'].between(index_period_start, index_period_end)].mean()

df = df.set_index('DATE')
df.index = df.index.astype('datetime64[ns]')

df = df/index_avg
print(df)

                  AU    AU-NSW     AU-NT    AU-QLD     AU-SA    AU-TAS  \
DATE                                                                     
2016-12-18  1.489971  1.289617  0.000000  0.651728  2.677686  3.723457   
2016-12-25  0.417192  0.218579  0.000000  0.454235  0.000000  0.000000   
2017-01-01  0.998281  0.699454  0.000000  0.789973  1.652893  0.000000   
2017-01-08  1.445272  1.551913  0.000000  1.579947  0.000000  0.000000   
2017-01-15  1.162178  0.939891  0.000000  1.342955  1.586777  3.209877   
...              ...       ...       ...       ...       ...       ...   
2021-11-07  1.087679  0.721311  4.549501  1.303456  1.487603  0.000000   
2021-11-14  1.102579  1.377049  0.000000  0.987467  1.454545  0.000000   
2021-11-21  1.370774  1.158470  0.000000  1.145461  1.950413  0.000000   
2021-11-28  0.908883  0.655738  0.000000  0.710976  0.000000  1.861728   
2021-12-05  0.879083  0.721311  2.474290  0.750475  0.528926  0.000000   

              AU-VIC     AU-WA       

In [15]:
df


,AU,AU-NSW,AU-NT,AU-QLD,AU-SA,AU-TAS,AU-VIC,AU-WA,CA,ES,FR,GB,GLOBAL,JP,KR,NZ,SG,US
DATE,,,,,,,,,,,,,,,,,,
2016-12-18,1.489971,1.289617,0.000000,0.651728,2.677686,3.723457,1.561952,0.550681,1.091654,0.575896,1.917266,1.045039,1.118188,1.970715,1.167290,0.708498,2.430117,1.020868
2016-12-25,0.417192,0.218579,0.000000,0.454235,0.000000,0.000000,0.802670,0.550681,0.985151,1.253420,1.309353,0.494527,0.870650,2.015504,1.222875,0.261025,0.270013,0.792422
2017-01-01,0.998281,0.699454,0.000000,0.789973,1.652893,0.000000,0.780976,2.045386,1.118280,1.727687,3.039568,1.222322,1.160867,0.000000,0.000000,0.689853,0.517525,1.063701
2017-01-08,1.445272,1.551913,0.000000,1.579947,0.000000,0.000000,1.627034,2.517398,1.291347,1.592182,2.291367,1.334290,1.348654,1.881137,0.555852,1.267838,1.237559,1.142230
2017-01-15,1.162178,0.939891,0.000000,1.342955,1.586777,3.209877,2.299541,1.494705,1.650794,1.558306,1.776978,1.474251,1.408404,1.881137,1.667557,0.820366,0.742536,1.292147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-07,1.087679,0.721311,4.549501,1.303456,1.487603,0.000000,0.368794,1.573374,1.184844,0.338762,0.374101,0.839763,0.973079,1.433247,0.416889,0.764432,1.147555,0.978034
2021-11-14,1.102579,1.377049,0.000000,0.987467,1.454545,0.000000,0.976220,1.573374,0.931900,0.982410,1.449640,0.998385,0.956008,0.716624,0.416889,0.130513,0.810039,0.935200
2021-11-21,1.370774,1.158470,0.000000,1.145461,1.950413,0.000000,0.737589,0.590015,0.918587,0.609772,2.151079,1.073031,0.819435,1.074935,1.278461,0.988168,0.810039,0.813839


In [26]:
help(fig.add_hline)

Help on method add_hline in module plotly.basedatatypes:

add_hline(y, row='all', col='all', exclude_empty_subplots=True, **kwargs) method of plotly.graph_objs._figure.Figure instance
    Add a horizontal line to a plot or subplot that extends infinitely in the
    x-dimension.
    
    Parameters
    ----------
    y: float or int
        A number representing the y coordinate of the horizontal line.
    exclude_empty_subplots: Boolean
        If True (default) do not place the shape on subplots that have no data
        plotted on them.
    row: None, int or 'all'
        Subplot row for shape indexed starting at 1. If 'all', addresses all rows in
        the specified column(s). If both row and col are None, addresses the
        first subplot if subplots exist, or the only plot. By default is "all".
    col: None, int or 'all'
        Subplot column for shape indexed starting at 1. If 'all', addresses all rows in
        the specified column(s). If both row and col are None, addres

In [46]:
# print(data)

import plotly.express as px
# import seaborn as sns
# sns.set_style('white')

roll_n = 52
df_plot = df.rolling(roll_n, min_periods=roll_n).mean()
df_plot = df_plot[df_plot.index>'2018-01-01']

fig = px.line(df_plot, x=df_plot.index, y=['AU', 'NZ', 'JP', 'US', 'GLOBAL'], title='Keyword Web Search Interest Over Time')
fig.add_hline(y=1, line_dash="dot",
              annotation_text="Index = 1 means average of Dec20-Nov21", 
              annotation_position="bottom right",
              annotation_font_size=10,
              annotation_font_color="blue") # add vertical line or rectangle https://plotly.com/python/horizontal-vertical-shapes/
fig.add_vline(x='2020-10-07', line_dash="dot")
fig.add_vrect(x0='2018-01-01', x1='2020-02-28'
                           ,annotation_text="pre Covid19", annotation_position="top left",
              annotation=dict(font_size=12, font_family="Calibri"),
              fillcolor="green", opacity=0.1, line_width=0)
fig.add_vrect(x0='2020-03-01', x1='2021-12-31'
                           ,annotation_text="Covid19", annotation_position="top left",
              annotation=dict(font_size=12, font_family="Calibri"),
              fillcolor="red", opacity=0.1, line_width=0)
fig.show() 

In [20]:
import plotly.express as px
# import seaborn as sns
# sns.set_style('white')

roll_n = 52
df_plot = df.rolling(roll_n, min_periods=roll_n).mean()
df_plot = df_plot[df_plot.index>'2018-01-01']

fig = px.line(df_plot, x=df_plot.index, y=['AU', 'AU-NSW', 'AU-VIC', 'AU-QLD'], title='Keyword Web Search Interest Over Time')
fig.show() 

## historical hourly interest

In [9]:
pytrends.get_historical_interest(kw_list, year_start=2021, month_start=9, day_start=1, hour_start=0, year_end=2021, month_end=9, day_end=30, hour_end=0, cat=0, sleep=0)


,machine learning,isPartial
date,,
2021-09-29 00:00:00,0,False
2021-09-29 01:00:00,0,False
2021-09-29 02:00:00,0,False
2021-09-29 03:00:00,0,False
2021-09-29 04:00:00,0,False
2021-09-29 05:00:00,0,False
2021-09-29 06:00:00,0,False
2021-09-29 07:00:00,0,False
2021-09-29 08:00:00,0,False


# Resources

Plotting using plotly

https://plotly.com/python/horizontal-vertical-shapes/